#Variational Quantum Eigensolver(VQE)、QAOAセミナー 2-7（中級設定編）

##2-7 中級設定
変分アルゴリズムを使い続けていると、様々な設定をしたくなってきます。今回は古典最適化部分を見てみたいと思います。

##2-7-1 古典最適化
変分アルゴリズムの内部構造は大きく分けて、

１、パラメータ付きの短い量子回路
２、量子回路を集計し、期待値を計算、次のパラメータの提案

です。次のトライアルのパラメータを渡すのは最適化アルゴリズムですが、それは古典で実行されます。

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 9.7MB/s 


In [2]:
import numpy as np
from blueqat import Circuit, vqe
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
minimizer=vqe.get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(h),minimizer=minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.999999802743004
Result by numpy
-8.0



上記では古典最適化のアルゴリズムを指定できています。主にscipyから選べます。

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP'

この辺りを見れば良さそうです。自分の好きなソルバーを組み込むことができます。

```python
def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=10000, trials=trials, verbose=1)
    return list(best.values())
```

##2-7-2 numbaバックエンドの採用
単純に高速化できます。元は、

In [0]:
from blueqat import BlueqatGlobalSetting
BlueqatGlobalSetting.set_default_backend('numpy')

In [11]:
import time
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

hamiltonian = q(0)*q(1)*q(2)*q(3)
step = 10

start = time.time()
result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run()
end = time.time()

print(end - start)

20.231501579284668


こちらはハミルトニアンを作ってvqeでstep = 10で実行しています。次にnumbaです。

In [0]:
from blueqat import BlueqatGlobalSetting
BlueqatGlobalSetting.set_default_backend('numba')

In [0]:
import time
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

hamiltonian = q(0)*q(1)*q(2)*q(3)
step = 10

start = time.time()
result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run()
end = time.time()

print(end - start)

速くなりました。

詳しくはこちらをみてください。
「blueqatでいろいろベンチマークとってみる。最適化とかGPUとか。」
https://qiita.com/YuichiroMinato/items/f08420957896b96e6874